<center>
<a href="http://www.insa-toulouse.fr/" ><img src="http://www.math.univ-toulouse.fr/~besse/Wikistat/Images/logo-insa.jpg" style="float:left; max-width: 120px; display: inline" alt="INSA"/></a> 

<a href="http://www.univ-tlse3.fr/" ><img src="http://www.univ-tlse3.fr/medias/photo/ut3pres_logoq_1372757033342.jpg?ID_FICHE=49702" style="float:right; max-width: 250px; display: inline" alt="INSA"/></a> 
</center>



# Biais et Discrimination en Apprentissage Statistique
## Détection et correction du biais  sur les données `Adult Income` avec <a href="https://www.python.org/"><img src="https://upload.wikimedia.org/wikipedia/commons/thumb/f/f8/Python_logo_and_wordmark.svg/390px-Python_logo_and_wordmark.svg.png" style="max-width: 120px; display: inline" alt="Python"/></a>

### Résumé
Utilisation des fonctions du [dépôt](https://github.com/algofairness/fairness-comparison) de  [Friedler et al. 2019](https://dl.acm.org/citation.cfm?id=3287589) pour tester différents outils de correction des biais d'apprentissage.

## Introduction
### Outils
Friedler et al. (2019) proposent une comparaison systématique de plusieurs algorithmes de réparation des données ou d'apprentissage loyal sur 5 jeux de données classiques: *Adult Income, Ricci, German Bank, Propublica recidivism, Propublica violent recidivism*. Dans la discussion finale, les auteurs insistent lourdement sur les choix opérés lors du prétraitement des données, la prolifération des mesures statistiques de biais et l'instabilité de la phase d'apprentissage. 
- Le prétraitement doit être rigoureusement identique avant l'application de différents algorithmes pour pouvoir en comparer les performances.
- Il suffit de se limiter à quelques mesures de biais car celles-ci sont fortement corrélées . Prendre par exemple en compte l'effet disproportionné (*disparate impact* ou *DI*) et une comparaison des taux d'erreur conditionnels.
- Il est important de reproduire les résultats sur plusieurs séparations aléatoires apprentissage - test des données.

    Friedler et al. (2019) analysent donc les résultats de 10 exécutions opérées sur 5 jeux de données et pour une liste de 19 algorithmes combinant différents apprentissages (naïf bayésien, régression logistique, SVM, arbre de décision) et correction (Calders, Zafar, Kamishima, Feldman) ou pas du biais. Par souci de reproductibilité de la recherche, le code python est disponible dans un [dépôt](https://github.com/algofairness/fairness-comparison) public. Il peut être installé avec la commande: 
    
    `pip3 install fairness`

### Objectif
*Attention* ce dépôt n'est pas une librairie comme peut l'être `scikit-learn` mais le code des programmes exécutant tous les prétraitements, les comparaisons de performances et la production des graphiques. En conséquence, l'exécution totale est excessivement longue et ne présente que peu d'intérêt. En revanche et dans l'attente de la réalisation d'une librairie efficace, il peut être utile ou intéressant de pouvoir mettre en oeuvre ces ressources comme pour produire les diagnostics et résultats d'un algorithme sur un jeu de données spécifique. C'est ce qui est tenté ci-dessous.

### Problèmes rencontrés
- Prétraitements: les auteurs ont à coeur de produire des codes génériques suffisamment généraux pour être exécutables sans intervention manuelle experte sur les données. C'est sûrement très positif pour une plus grande automatisation et standadisation des codes mais ne corrige pas des incohérences, erreurs de codification... pouvant être présentes et en fait toujours présentes dans des données réelles. D'autre part ils ne procèdent pas à des regroupements de modalités, par exemple en lien avec l'origine des individus. Certaines variables, dont celle sensible d'origine, possèdent un nombre important de modalités avec certaines de faible effectif donc sans réel intérêt. Cela complique inutilement les traitements et la production de résultats.
- Ce sont des codes itératifs exécutant sytématiquement toutes les combinaisons des paramètres. Exécution très longue, il serait préférable de pouvoir faire des choix (données algorithme, paramètre) mais les fonctions ne sont pas documentées et donc difficilement adaptables à des besoins spécifiques.
- Problème de mise à jour et compatibilité de certaines librairies entraînant la production de nombreux *warning* et des erreurs d'exécution pour certains algorithmes; tous les résultats ne peuvent pas être reproduits.
- Volume des résultats: les objectifs de l'article conduivent à calculer tous les critères possibles pour finalement montrer que ceux-ci sont très liés et que quelques uns suffisent à caractériser les biais et la qualité de prévision; extraire les plus pertinents des fichiers de sortie n'est pas si simple.

#### Questions
- Il reste une incohérence à expliquer entre les résultats de ce calepin avec les résultats produits par le calepin en R sur les mêmes données. En moyenne, l'accroissement du biais de la précison est plus important lorsque les algorithmes sont exécutés en R (20 itérations) plutôt qu'en Python (10 itérations). Cela concerne les arbres de décision et la régression logistique. La seule différence évidente entre les deux codes concerne le pré-traitement des données; manuel en R automatique en Python. 
- A faire: récupérer les données transformées issues du dépôt python pour les traiter en R.
- Les traitements longs et complexes de réparation des données ou d'apprentissage sous contrainte de loyauté sont-ils justifiés? Ne suffit-il pas de corriger naïvement, biaiser en faveur du groupe défavorisé, le seuil de la prise de décision de l'algorithme d'apprentissage retenu? Cf. Calders et Verwer (2010) pour le classifieur bayésien naïf. 

#### Conclusion
Une fois ces tâches de comparaison réalisées et les bons outils sélectionnés, il serait pertinent de produire une librairie efficace car parallélisée et industrialisable de correction du biais pour aborder des problèmes de la vraie vie et pas seulement des jeux de données publics mais élémentaires.


Les codes exécutés ci-dessous sont extraits du [dépôt](https://github.com/algofairness/fairness-comparison) associé à  l'article de Friedler et al. (2019).
## Prétraitements
Les codes ci-dessous permettent de transformer les données initiales des 5 fichiers pour les rendre compatibles avec les traitements suivants. il s'agit avant tout de recoder les variables qualitatives, éventuellement celles quantitatives et de supprimer toutes les observations avec des données manquantes.
#### Librairies

In [1]:
import sys
import os
import pandas as pd
import fire

In [3]:
from fairness.data.objects.list import DATASETS, get_dataset_names

fairness.data.objects.Adult.Adult

#### Choix d'un seul jeu de données
Le process n'est appliqué qu'au jeude données `adult income` à titre d'illustration. De toute façon, les données tranformées sont enregistrées sans le dépôt.

In [4]:
DATASETS[1]

In [5]:
def prepare_data(dataset_names = get_dataset_names()):

    for dataset in DATASETS:
        if not dataset.get_dataset_name() in dataset_names:
            continue
        print("--- Processing dataset: %s ---" % dataset.get_dataset_name())
        data_frame = dataset.load_raw_dataset()
        d = preprocess(dataset, data_frame)
        
        for k, v in d.items():
            write_to_file(dataset.get_filename(k), v)

def write_to_file(filename, dataframe):
    print("Writing data to: %s" % filename)
    dataframe.to_csv(filename, index = False)

def preprocess(dataset, data_frame):
    """
    The preprocess function takes a pandas data frame and returns two modified data frames:
    1) all the data as given with any features that should not be used for training or fairness
    analysis removed.
    2) only the numerical and ordered categorical data, sensitive attributes, and class attribute.
    Categorical attributes are one-hot encoded.
    3) the numerical data (#2) but with a binary (numerical) sensitive attribute
    """

    # Remove any columns not included in the list of features to keep.
    smaller_data = data_frame[dataset.get_features_to_keep()]

    # Handle missing data.
    missing_processed = dataset.handle_missing_data(smaller_data)

    # Remove any rows that have missing data.
    missing_data_removed = missing_processed.dropna()
    missing_data_count = missing_processed.shape[0] - missing_data_removed.shape[0]
    if missing_data_count > 0:
        print("Missing Data: " + str(missing_data_count) + " rows removed from dataset " +  \
              dataset.get_dataset_name())

    # Do any data specific processing.
    processed_data = dataset.data_specific_processing(missing_data_removed)

    print("\n-------------------")
    print("Balance statistics:")
    print("\nClass:")
    print(dataset.get_class_balance_statistics(processed_data))
    print("\nSensitive Attribute:")
    for r in dataset.get_sensitive_attribute_balance_statistics(processed_data):
        print(r)
        print("\n")
    print("\n")

    # Handle multiple sensitive attributes by creating a new attribute that's the joint distribution
    # of all of those attributes.  For example, if a dataset has both 'Race' and 'Gender', the
    # combined feature 'Race-Gender' is created that has attributes, e.g., 'White-Woman'.
    sensitive_attrs = dataset.get_sensitive_attributes()
    if len(sensitive_attrs) > 1:
        new_attr_name = '-'.join(sensitive_attrs)
        ## TODO: the below may fail for non-string attributes
        processed_data = processed_data.assign(temp_name =
                             processed_data[sensitive_attrs].apply('-'.join, axis=1))
        processed_data = processed_data.rename(columns = {'temp_name' : new_attr_name})
        # dataset.append_sensitive_attribute(new_attr_name)
        # privileged_joint_vals = '-'.join(dataset.get_privileged_class_names(""))
        # dataset.get_privileged_class_names("").append(privileged_joint_vals)

    # Create a one-hot encoding of the categorical variables.
    processed_numerical = pd.get_dummies(processed_data,
                                         columns = dataset.get_categorical_features())

    # Create a version of the numerical data for which the sensitive attribute is binary.
    sensitive_attrs = dataset.get_sensitive_attributes_with_joint()
    privileged_vals = dataset.get_privileged_class_names_with_joint("")
    processed_binsensitive = make_sensitive_attrs_binary(
        processed_numerical, sensitive_attrs, privileged_vals)

    # Create a version of the categorical data for which the sensitive attributes is binary.
    processed_categorical_binsensitive = make_sensitive_attrs_binary(
        processed_data, sensitive_attrs,
        dataset.get_privileged_class_names("")) ## FIXME
    # Make the class attribute numerical if it wasn't already (just for the bin_sensitive version).
    class_attr = dataset.get_class_attribute()
    pos_val = dataset.get_positive_class_val("") ## FIXME

    processed_binsensitive = make_class_attr_num(processed_binsensitive, class_attr, pos_val)

    return { "original": processed_data,
             "numerical": processed_numerical,
             "numerical-binsensitive": processed_binsensitive,
             "categorical-binsensitive": processed_categorical_binsensitive }

def make_sensitive_attrs_binary(dataframe, sensitive_attrs, privileged_vals):
    newframe = dataframe.copy()
    for attr, privileged in zip(sensitive_attrs, privileged_vals):
        # replace privileged vals with 1
        newframe[attr] = newframe[attr].replace({ privileged : 1 })
        # replace all other vals with 0
        newframe[attr] = newframe[attr].replace("[^1]", 0, regex = True)
    return newframe

def make_class_attr_num(dataframe, class_attr, positive_val):
    # don't change the class attribute unless its a string (pandas type: object)
    if (dataframe[class_attr].dtypes == 'object'):
        dataframe[class_attr] = dataframe[class_attr].replace({ positive_val : 1 })
        dataframe[class_attr] = dataframe[class_attr].replace("[^1]", 0, regex = True)
    return dataframe


In [6]:
dataset=DATASETS[1]
data_frame = dataset.load_raw_dataset()
d = preprocess(dataset, data_frame)

Missing Data: 2399 rows removed from dataset adult

-------------------
Balance statistics:

Class:
income-per-year
<=50K    22654
>50K      7508
dtype: int64

Sensitive Attribute:
race
Amer-Indian-Eskimo      286
Asian-Pac-Islander      895
Black                  2817
Other                   231
White                 25933
dtype: int64


sex
Female     9782
Male      20380
dtype: int64






Attention, les données transformées sont sauvegardées par défaut dans l'architecture locale du dépôt.

In [25]:
for k, v in d.items():
            write_to_file(dataset.get_filename(k), v)

Writing data to: /home-local/pbesse/anaconda3/lib/python3.6/site-packages/fairness/data/preprocessed/adult_original.csv
Writing data to: /home-local/pbesse/anaconda3/lib/python3.6/site-packages/fairness/data/preprocessed/adult_numerical.csv
Writing data to: /home-local/pbesse/anaconda3/lib/python3.6/site-packages/fairness/data/preprocessed/adult_numerical-binsensitive.csv
Writing data to: /home-local/pbesse/anaconda3/lib/python3.6/site-packages/fairness/data/preprocessed/adult_categorical-binsensitive.csv


## Exécution
#### Librairie et fonctions

In [22]:
import fire
import os
import statistics
import sys

from fairness import results
from fairness.data.objects.list import DATASETS, get_dataset_names
from fairness.data.objects.ProcessedData import ProcessedData
from fairness.algorithms.list import ALGORITHMS
from fairness.metrics.list import get_metrics

from fairness.algorithms.ParamGridSearch import ParamGridSearch

NUM_TRIALS_DEFAULT = 10

def get_algorithm_names():
    result = [algorithm.get_name() for algorithm in ALGORITHMS]
    print("Available algorithms:")
    for a in result:
        print("  %s" % a)
    return result

def run(num_trials = NUM_TRIALS_DEFAULT, dataset = get_dataset_names(),
        algorithm = get_algorithm_names()):
    algorithms_to_run = algorithm

    print("Datasets: '%s'" % dataset)
    for dataset_obj in DATASETS:
        if not dataset_obj.get_dataset_name() in dataset:
            continue

        print("\nEvaluating dataset:" + dataset_obj.get_dataset_name())

        processed_dataset = ProcessedData(dataset_obj)
        train_test_splits = processed_dataset.create_train_test_splits(num_trials)

        all_sensitive_attributes = dataset_obj.get_sensitive_attributes_with_joint()
        for sensitive in all_sensitive_attributes:

            print("Sensitive attribute:" + sensitive)

            detailed_files = dict((k, create_detailed_file(
                                          dataset_obj.get_results_filename(sensitive, k),
                                          dataset_obj,
                                          processed_dataset.get_sensitive_values(k), k))
                for k in train_test_splits.keys())

            for algorithm in ALGORITHMS:
                if not algorithm.get_name() in algorithms_to_run:
                    continue

                print("    Algorithm: %s" % algorithm.get_name())
                print("       supported types: %s" % algorithm.get_supported_data_types())
                if algorithm.__class__ is ParamGridSearch:
                    param_files =  \
                        dict((k, create_detailed_file(
                                     dataset_obj.get_param_results_filename(sensitive, k,
                                                                            algorithm.get_name()),
                                     dataset_obj, processed_dataset.get_sensitive_values(k), k))
                          for k in train_test_splits.keys())
                for i in range(0, num_trials):
                    for supported_tag in algorithm.get_supported_data_types():
                        train, test = train_test_splits[supported_tag][i]
                        try:
                            params, results, param_results =  \
                                run_eval_alg(algorithm, train, test, dataset_obj, processed_dataset,
                                             all_sensitive_attributes, sensitive, supported_tag)
                        except Exception as e:
                            import traceback
                            traceback.print_exc(file=sys.stderr)
                            print("Failed: %s" % e, file=sys.stderr)
                        else:
                            write_alg_results(detailed_files[supported_tag],
                                              algorithm.get_name(), params, i, results)
                            if algorithm.__class__ is ParamGridSearch:
                                for params, results in param_results:
                                    write_alg_results(param_files[supported_tag],
                                                      algorithm.get_name(), params, i, results)

            print("Results written to:")
            for supported_tag in algorithm.get_supported_data_types():
                print("    %s" % dataset_obj.get_results_filename(sensitive, supported_tag))

            for detailed_file in detailed_files.values():
                detailed_file.close()

def write_alg_results(file_handle, alg_name, params, run_id, results_list):
    line = alg_name + ','
    params = ";".join("%s=%s" % (k, v) for (k, v) in params.items())
    line += params + (',%s,' % run_id)
    line += ','.join(str(x) for x in results_list) + '\n'
    file_handle.write(line)

def run_eval_alg(algorithm, train, test, dataset, processed_data, all_sensitive_attributes,
                 single_sensitive, tag):
    """
    Runs the algorithm and gets the resulting metric evaluations.
    """
    privileged_vals = dataset.get_privileged_class_names_with_joint(tag)
    positive_val = dataset.get_positive_class_val(tag)

    # get the actual classifications and sensitive attributes
    actual = test[dataset.get_class_attribute()].values.tolist()
    sensitive = test[single_sensitive].values.tolist()

    predicted, params, predictions_list =  \
        run_alg(algorithm, train, test, dataset, all_sensitive_attributes, single_sensitive,
                privileged_vals, positive_val)

    # make dictionary mapping sensitive names to sensitive attr test data lists
    dict_sensitive_lists = {}
    for sens in all_sensitive_attributes:
        dict_sensitive_lists[sens] = test[sens].values.tolist()

    sensitive_dict = processed_data.get_sensitive_values(tag)
    one_run_results = []
    for metric in get_metrics(dataset, sensitive_dict, tag):
        result = metric.calc(actual, predicted, dict_sensitive_lists, single_sensitive,
                             privileged_vals, positive_val)
        one_run_results.append(result)

    # handling the set of predictions returned by ParamGridSearch
    results_lol = []
    if len(predictions_list) > 0:
        for param_name, param_val, predictions in predictions_list:
            params_dict = { param_name : param_val }
            results = []
            for metric in get_metrics(dataset, sensitive_dict, tag):
                result = metric.calc(actual, predictions, dict_sensitive_lists, single_sensitive,
                                     privileged_vals, positive_val)
                results.append(result)
            results_lol.append( (params_dict, results) )

    return params, one_run_results, results_lol

def run_alg(algorithm, train, test, dataset, all_sensitive_attributes, single_sensitive,
            privileged_vals, positive_val):
    class_attr = dataset.get_class_attribute()
    params = algorithm.get_default_params()

    # Note: the training and test set here still include the sensitive attributes because
    # some fairness aware algorithms may need those in the dataset.  They should be removed
    # before any model training is done.
    predictions, predictions_list =  \
        algorithm.run(train, test, class_attr, positive_val, all_sensitive_attributes,
                      single_sensitive, privileged_vals, params)

    return predictions, params, predictions_list


def get_dict_sensitive_vals(dict_sensitive_lists):
    """
    Takes a dictionary mapping sensitive attributes to lists in the test data and returns a
    dictionary mapping sensitive attributes to lists containing each sensitive value only once.
    """
    newdict = {}
    for sens in dict_sensitive_lists:
         sensitive = dict_sensitive_lists[sens]
         newdict[sens] = list(set(sensitive))
    return newdict

def create_detailed_file(filename, dataset, sensitive_dict, tag):
    return results.ResultsFile(filename, dataset, sensitive_dict, tag)
    # f = open(filename, 'w')
    # f.write(get_detailed_metrics_header(dataset, sensitive_dict, tag) + '\n')
    # return f


Available algorithms:
  SVM
  GaussianNB
  LR
  DecisionTree
  Kamishima
  Calders
  ZafarBaseline
  ZafarFairness
  ZafarAccuracy
  Kamishima-accuracy
  Kamishima-DIavgall
  Feldman-SVM
  Feldman-GaussianNB
  Feldman-LR
  Feldman-DecisionTree
  Feldman-SVM-DIavgall
  Feldman-SVM-accuracy
  Feldman-GaussianNB-DIavgall
  Feldman-GaussianNB-accuracy


#### Liste des algorithmes disponibles
Il ya ceux classiques d'apprentissage sans correction de biais (base line) et ceux opérant par loyauté par une transformation des données, ou en contraignant l'étape d'apprentissage. 

In [29]:
get_algorithm_names()[5]

Available algorithms:
  SVM
  GaussianNB
  LR
  DecisionTree
  Kamishima
  Calders
  ZafarBaseline
  ZafarFairness
  ZafarAccuracy
  Kamishima-accuracy
  Kamishima-DIavgall
  Feldman-SVM
  Feldman-GaussianNB
  Feldman-LR
  Feldman-DecisionTree
  Feldman-SVM-DIavgall
  Feldman-SVM-accuracy
  Feldman-GaussianNB-DIavgall
  Feldman-GaussianNB-accuracy


'Calders'

#### Exécution
`num_trials` exécutions pour un jeu de données et un algorithme déterminés. Les résultats sont empilés dans un ensemble de fichiers. 

**Attention** Certains algorithmes (Calders) plantent alors que d'autres (Feldman-SVM-DIavgall) sont excessivement longs. La cellule ci-dessous  a été exécutée pour différents algorihtmes; cf. ci-dessous.

In [37]:
run(num_trials = 2, dataset = 'adult',algorithm = 'DecisionTree')

Datasets: 'adult'

Evaluating dataset:adult
Sensitive attribute:race
    Algorithm: DecisionTree
       supported types: {'numerical-binsensitive', 'numerical'}
Results written to:
    /home-local/pbesse/.fairness/results/adult_race_numerical-binsensitive.csv
    /home-local/pbesse/.fairness/results/adult_race_numerical.csv
Sensitive attribute:sex
    Algorithm: DecisionTree
       supported types: {'numerical-binsensitive', 'numerical'}
Results written to:
    /home-local/pbesse/.fairness/results/adult_sex_numerical-binsensitive.csv
    /home-local/pbesse/.fairness/results/adult_sex_numerical.csv
Sensitive attribute:race-sex
    Algorithm: DecisionTree
       supported types: {'numerical-binsensitive', 'numerical'}
Results written to:
    /home-local/pbesse/.fairness/results/adult_race-sex_numerical-binsensitive.csv
    /home-local/pbesse/.fairness/results/adult_race-sex_numerical.csv


## Résultats
Tous les résultats intermédiaires sont stockés dans des fichiers archivés dans un répertoire masqué. Ces fichiers sont destinés à être lus par le programme `results.py` capable d'en extraire les valeurs utiles au calcul d'une liste très exhaustive de métriques. Ces derniers résultats sont également stockés puit l'exécution du programme `analysis.py` fournit les graphiques de l'article à l'aide de fonctions R (`ggplot`). 

Il s'agit donc, dans ce calepin, d'extraire de ces fichiers à titre d'exemple les indicateurs plus pertinents sans souci d'exhaustivité. *Soucis*: les intitulés ne sont pas très explicites et le nombre de modalités de la variable d'origine ethnique fait exploser la combinatoire de résultats possibles. Il semble important de se limiter à la seule prise en compte de variables sensibles binaires: genre ou origine ethnique caucasien *vs.* non caucasien. La prise en compte des interactions entre les deux variables sensibles introduit également une forte complexité pas indispensables en première lecture.

Voici trois colonnes extraites des fichiers: `adult_sex_numerical-binsensitive.csv` et `adult_race_numerical-binsensitive.csv` qui en comporte 2 * 127 lorsque les variables sensibles sont considérées binaires. Dans le cas contraire, si toutes les modalités d'origine sont prises en compte, le fichier `adult_sex_numerical.csv` comporte 389 colonnes!



|Algorithme | Paramètre | DI - genre| DI - origine|
|----------|--------|-------|----|
|SVM| none | 0.25|0.62|
|SVM| none | 0.28|0.64|
|GaussianNB| none | 0.32|0.62|
|GaussianNB| none | 0.34|0.60|
|LR | none | 0.33 |0.61|
|LR | none | 0.30|0.61|
|DecisionTree|none|0.44|0.70|
|DecisionTree|none|0.43|0.60|
|Kamishima|eta=1.0| 0.28|0.53|
|Kamishima|eta=1.0|0.30|0.58|
|ZafarFairness|	c=0.001	| 0.29|0.50|
|ZafarFairness|	c=0.001	| 0.29|0.53|
|Kamishima-DIavgall|eta=1.0| 0.28|0.53|
|Kamishima-DIavgall|eta=1.0|0.30|0.58|
|Feldman-SVM|lambda=1.0|0.30|0.68|
|Feldman-SVM|lambda=1.0|0.37|0.62|
|Feldman-SVM-DIavgall|lambda=0.35|0.30|0.81|
|Feldman-SVM-DIavgall|lambda=0.45|0.35|0.68|

**Remarques**
- Compte tenu que seulement 2 exécutions ont été réalisées, les résultats semblent cohérents avec les graphiques de l'article (figure 6 page 16).
- En revanche, les *DI* obtenus pour les algorithmes d'apprentissage seuls ne sont pas cohérents avec ceux calculés en R. A contrôler: est-ce dû aux différences entre les procédures de prétraitement?
- Les corrections de biais vis-à-vis du genre ne sont pas très probantes pour les couples algo x paramètre exécutés par rapports aux algorithmes seuls qui constituent des *base lines*. C'est explicable vis-à-vis de la variable genre car l'optimisation des paramètres a sans doute été réalisée avec pour objectif de réduire le biais liée à la variable origine. Néanmoins même dans ce cas, cette réduction n'est pas exceptionnelle.
- Il faudrait aussi ajouter la précision et choisir un indicateur de taux d'erreur conditionnel.

## Comparaison
Comparer avec les résultats produits de façon plus simple voire naïve dans R:
- calculer sur le fichier issu du prétraitement le *DI* d'origine: `genre` *vs.* `income`,
- calculer la prévision par régression logistique,
- calculer le *DI*,
- faire la correciton élémentaire du seuil de décision.

## Références
Calders T., Verwer S. (2010). Three naive Bayes approaches for discrimination-free classification, Data Mining and Knowledge Discovery, 21 (2), pp 277–292.

Friedler S., Scheidegger C., Venkatasubramanian S., Choudhary S., Hamilton E., Roth D. (2019). [A comparative study of fairness-enhancing interventions in machine learning](https://dl.acm.org/citation.cfm?id=3287589), Proceedings of the Conference on Fairness, Accountability, and Transparency.